In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.executable

'/home/junwoony/miniconda3/envs/schnet2/bin/python'

In [3]:
sys.path

['/home/junwoony/Desktop/surface_seg/examples',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python37.zip',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/lib-dynload',
 '',
 '/home/junwoony/.local/lib/python3.7/site-packages',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/site-packages',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/site-packages/PyDispatcher-2.0.5-py3.5.egg',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/site-packages/torch_geometric-1.3.2-py3.7.egg',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/site-packages/joblib-0.14.1.dev0-py3.7.egg',
 '/home/junwoony/Desktop/surface_seg',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/site-packages/palettable-3.1.1-py3.7.egg',
 '/home/junwoony/miniconda3/envs/schnet2/lib/python3.7/site-packages/IPython/extensions',
 '/home/junwoony/.ipython']

Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [4]:
from surface_seg.envs.mcs_env import MCSEnv
import gym.wrappers

MCS_gym = MCSEnv()
MCS_gym = gym.wrappers.Monitor(MCS_gym, "./vid", force=True)

print('Starting energy = %1.2f eV'%MCS_gym.atoms.get_potential_energy())

Starting energy = 2.49 eV


/home/junwoony/.local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/junwoony/.local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Set up the gym and agent in tensorforce

In [5]:
import tensorforce 
from tensorforce.agents import Agent


env = tensorforce.environments.OpenAIGym(MCS_gym,
                                                 max_episode_timesteps=200,
                                                visualize=False)

agent = Agent.create(
    agent='dqn', 
    environment=env, 
    batch_size=10, 
    learning_rate=1e-3,
    memory = 40000,
    max_episode_timesteps = 200,
    exploration=0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Run the DRL method for a while

In [11]:
from tensorforce.execution import Runner

runner = Runner(
    agent=agent,
    environment=env,
    max_episode_timesteps=200,
)

runner.run(num_episodes=1000)

# runner.run(num_episodes=10, evaluation=True)

runner.close()



Episodes:   0%|          | 0/1000 [00:00, reward=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%]

Episodes:   0%|          | 1/1000 [00:07, reward=-2.47, ts/ep=137, sec/ep=7.99, ms/ts=58.3, agent=35.7%]

Episodes:   0%|          | 2/1000 [00:17, reward=-3.09, ts/ep=175, sec/ep=9.66, ms/ts=55.2, agent=34.3%]

KeyboardInterrupt: 

# Example of how to render the environment in a jupyter cell

In [ ]:
env = MCS_gym
from matplotlib import pyplot as plt
%matplotlib inline

from IPython import display

img = plt.imshow(env.render(mode='rgb_array')) # only call this once
plt.axis('off')
for _ in range(100):
    img.set_data(env.render(mode='rgb_array')) # just update the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    action = env.action_space.sample()
    env.step(action)